connecting

In [ ]:
%matplotlib inline
import logging
import qcodes as qc
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB20,
    RohdeSchwarzZNBChannel,
)
from qcodes.dataset import (
    Measurement,
    initialise_database,
    load_or_create_experiment,
    plot_by_id,
)

initialise_database()
load_or_create_experiment(experiment_name='test_with_znb20',
                          sample_name="no sample")
vna = RohdeSchwarzZNB20('VNA', 'TCPIP0::192.168.50.249::inst0::INSTR')
station = qc.Station(vna)
vna.clear_channels()
vna.add_channel('S43')

find the range of scaning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, peak_widths
from qcodes.dataset.plotting import plot_by_id

# 設置掃描參數
vna.channels.S43.npts(600)
vna.channels.avg(1)
vna.channels.S43.bandwidth(1e2)
vna.channels.S43.power(-20)
vna.rf_on()
#粗估resonator頻率位置
input_frequencies = [4.364e9, 4.442e9, 4.520e9, 4.894e9, 4.977e9, 5.061e9, 5.266e9, 5.351e9, 5.436e9, 5.585e9, 5.670e9, 5.756e9]
#粗估頻率掃描範圍
deltafre = 5e6

def measure_and_find_peak(vna, meas):
    with meas.run() as datasaver:
        get_v = vna.channels.S43.trace_db_phase.get()
        datasaver.add_result((vna.channels.S43.trace_db_phase, get_v))
    ax, cbax = plot_by_id(datasaver.run_id)

    frequencies = np.linspace(vna.channels.S43.start(), vna.channels.S43.stop(), len(get_v[0]))
    magnitude = get_v[0]

    peaks, _ = find_peaks(-magnitude)  

    if len(peaks) > 0:
        peak_index = peaks[np.argmin(magnitude[peaks])]
        peak_frequency = frequencies[peak_index]

        results_half = peak_widths(-magnitude, [peak_index], rel_height=0.5)
        fwhm = results_half[0][0] * (frequencies[1] - frequencies[0])

        new_start = peak_frequency - 9 * fwhm
        new_stop = peak_frequency + 9 * fwhm

        return new_start, new_stop
    else:
        raise ValueError("未找到向下的峰")

resonatorfre = []

for freq in input_frequencies:
    start_freq = freq - deltafre
    stop_freq = freq + deltafre
    
    vna.channels.S43.start(start_freq)
    vna.channels.S43.stop(stop_freq)

    meas = Measurement()
    meas.register_parameter(vna.channels.S43.trace_db_phase)
    try:
        new_start, new_stop = measure_and_find_peak(vna, meas)
        vna.channels.S43.start(new_start)
        vna.channels.S43.stop(new_stop)

        new_start, new_stop = measure_and_find_peak(vna, meas)
        vna.channels.S43.start(new_start)
        vna.channels.S43.stop(new_stop)

        resonatorfre.append([new_start, new_stop])
    except ValueError as e:
        print(f"在頻率 {freq} 附近未找到向下的峰：{e}")

for i, (new_start, new_stop) in enumerate(resonatorfre):
    print(f"輸入頻率 {input_frequencies[i]} 的新頻率範圍：{new_start} - {new_stop}")

In [ ]:
print(resonatorfre)

scaning

In [ ]:
import os
import json
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from qcodes.dataset.measurements import Measurement

powers = np.linspace(-60, 0, 13)

main_dir = r"C:\Users\ASUS\Documents\python training\測試 最終板\raw"  # 記得修改路徑
os.makedirs(main_dir, exist_ok=True)

for i in resonatorfre:

    [startfrequency, stopfrequency] = i
    points = 601
    vna.channels.S43.start(startfrequency)
    vna.channels.S43.stop(stopfrequency)
    vna.channels.S43.npts(points)
    #ifbw跟平均次數可改動
    vna.channels.S43.bandwidth(10)
    vna.channels.avg(1)

    frequencies = np.linspace(startfrequency, stopfrequency, points)  

    data_dict = {
        'ZZA': [],
        'ZZP': [],
        'ZZI': [],
        'ZZQ': [],
        'x': powers,
        'y': frequencies / 1e9,  
        'xtitle': 'Power(dBm)',
        'ytitle': 'Frequency(GHz)'
    }

    vna.rf_on()

    for power in powers:
        vna.channels.S43.power(power)
        
        meas = Measurement()
        meas.register_parameter(vna.channels.S43.trace_db_phase)
        
        with meas.run() as datasaver:

            magnitude_phase_data = vna.channels.S43.trace_db_phase.get()
            magnitudes_db = magnitude_phase_data[0]
            phases = magnitude_phase_data[1]
            magnitudes_linear = 10**(magnitudes_db/20)

            real_parts = magnitudes_linear * np.cos(phases)
            imaginary_parts = magnitudes_linear * np.sin(phases)
            
            datasaver.add_result((vna.channels.S43.trace_db_phase, magnitude_phase_data))
            run_id = datasaver.run_id
            
            data_dict['ZZA'].append(magnitudes_db)
            data_dict['ZZP'].append(phases)
            data_dict['ZZI'].append(real_parts)
            data_dict['ZZQ'].append(imaginary_parts)

    data_dict['ZZA'] = np.transpose(np.array(data_dict['ZZA']))
    data_dict['ZZP'] = np.transpose(np.array(data_dict['ZZP']))
    data_dict['ZZI'] = np.transpose(np.array(data_dict['ZZI']))
    data_dict['ZZQ'] = np.transpose(np.array(data_dict['ZZQ']))
    data_dict['x'] = np.array(data_dict['x'])
    data_dict['y'] = np.array(data_dict['y'])

    folder_name = f"C{int((startfrequency + stopfrequency) / 2 * 1e6):010.4f}"[:5]
    folder_path = os.path.join(main_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    mat_file_name = f"high_power_dependence{(startfrequency + stopfrequency) / 2}.mat"
    mat_file_path = os.path.join(folder_path, mat_file_name)

    sio.savemat(mat_file_path, data_dict)

    config_file_path = os.path.join(folder_path, "config.json")
    if os.path.exists(config_file_path):
        with open(config_file_path, 'r') as config_file:
            config_data = json.load(config_file)
    else:
        config_data = []

    new_config_entry = {
        "file_name": mat_file_name,
        "attenuation": 80  # 記得改衰減值
    }
    config_data.append(new_config_entry)

    with open(config_file_path, 'w') as config_file:
        json.dump(config_data, config_file, indent=4)

    # power dependence 的2D圖(可有可無)
    plt.figure(figsize=(10, 8))
    plt.contourf(data_dict['x'], data_dict['y'], data_dict['ZZA'], cmap='viridis')
    plt.colorbar(label='Magnitude (dB)')
    plt.title('Power Dependence of Magnitude (ZZA)')
    plt.xlabel('Power(dBm)')
    plt.ylabel('Frequency(GHz)')
    plt.show()


In [4]:
import os
import json
import xarray as xr
import numpy as np
import matplotlib as plt
from datetime import datetime
from qcat.utility.file_structure import *
from qcat.utility.io_translator import *


In [ ]:
from qcat.utility.file_structure import create_subfolder


raw_data_fd = r"C:\Users\ASUS\Documents\python training\測試 最終板\raw"
output_fd = r"C:\Users\ASUS\Documents\python training\測試 最終板\preprocess"
# Get all resonator from folder
resonator_list = [d for d in os.listdir(raw_data_fd) if os.path.isdir(os.path.join(raw_data_fd, d))]

print(resonator_list)

for resonator_label in resonator_list:

    print(f"{resonator_label}")
    resonator_fd = f"{raw_data_fd}\\{resonator_label}"
    power_list = [d for d in os.listdir(resonator_fd) if os.path.isdir(os.path.join(resonator_fd, d))]

    create_subfolder(output_fd,resonator_label)
    ave_resonator_folder = f"{output_fd}\\{resonator_label}" 

    mat_files = check_file_extension( resonator_fd, "mat")
    df_config = pd.read_json(f'{resonator_fd}\\config.json')
    for index, row in df_config.iterrows():
        attenuation = row["attenuation"]
        file_name = row["file_name"]
        print(f"{file_name} with {attenuation} dB attenuation")
        power_list, frequency, s21 = mat_to_numpy(f"{resonator_fd}\\{file_name}")
        print(frequency[0])
        zdata_2d = s21.transpose()
        for i, power_label in enumerate(power_list):
            # print(power_label)
            data = zdata_2d[i]


            # Creating an xarray dataset
            output_data = {
                "s21": ( ["s_params","frequency"],
                        np.array([data.real, data.imag]) )
            }
            dataset = xr.Dataset(
                output_data,
                coords={ "s_params":np.array(["real","imag"]), "frequency": frequency*1e9 })
            
            dataset.attrs["attenuation"] = int(attenuation)
            dataset.attrs["power"] = power_label
            dataset.to_netcdf(f"{ave_resonator_folder}\\PYQUM_{str(attenuation)}_{str(power_label)}.nc")


In [ ]:
from qcat.analysis.resonator.photon_dep.res_data import *
from qcat.utility.file_structure import check_file_extension, create_subfolder


raw_data_fd = r"C:\Users\ASUS\Documents\python training\thinfat\preprocess"
output_fd = r"C:\Users\ASUS\Documents\python training\thinfat\result"
# Get all resonator from folder
resonator_list = [d for d in os.listdir(raw_data_fd) if os.path.isdir(os.path.join(raw_data_fd, d))]

# check_configure(f"{output_fd}", ["power_dep_fit"])

all_resonator_result = []

for resonator_label in resonator_list:
    print(f"Processing {resonator_label}")

    create_subfolder(output_fd,resonator_label)
    result_folder = f"{output_fd}\\{resonator_label}"

    resonator_data_folder = f"{raw_data_fd}\\{resonator_label}"
    resonator = PhotonDepResonator(resonator_label)
    # Find cavity data (nc file) in the folder
    file_list = check_file_extension( resonator_data_folder, "nc")
    print(resonator_data_folder)

    for f_name in file_list:
        dataset = xr.open_dataset(f"{resonator_data_folder}\\{f_name}")
        power = dataset.attrs["power"]
        attenuation = dataset.attrs["attenuation"]
        frequency = dataset.coords["frequency"].values
        data = dataset["s21"].values
        resonator.import_array(frequency, data[0]+1j*data[1], power-attenuation)
    result = resonator.refined_analysis( result_folder )
    # result = resonator.free_analysis( result_folder )

    all_resonator_result.append( result )
    
    df_results = pd.concat(all_resonator_result)
    df_results.Name = resonator_label



In [ ]:
plot_data_fd = r"C:\Users\ASUS\Documents\python training\thinfat\result"
output_fd = r"C:\Users\ASUS\Documents\python training\thinfat\result"
from qcat.visualization.photon_dep_loss import * 
all_resonator_result = []
folder_list = [d for d in os.listdir(raw_data_fd) if os.path.isdir(os.path.join(raw_data_fd, d))]

fr = []
qc = []
for resonator_label in folder_list:
    create_subfolder(output_fd,resonator_label)
    result_folder = f"{output_fd}\\{resonator_label}"

    # Plotting
    df_powerQ_free = pd.read_csv( f"{plot_data_fd}\\{resonator_label}\\free_result.csv" )
    plot_singleRes_powerQ_free(df_powerQ_free, cav_label=f"{resonator_label}", output_fd=result_folder)
    plot_singleRes_powerloss_free(df_powerQ_free, cav_label=f"{resonator_label}", output_fd=result_folder)

    df_powerQ_refined = pd.read_csv( f"{plot_data_fd}\\{resonator_label}\\refined_result.csv" )
    plot_singleRes_powerQ_refined(df_powerQ_refined, cav_label=f"{resonator_label}", output_fd=result_folder)
    plot_singleRes_powerloss_refined(df_powerQ_refined, cav_label=f"{resonator_label}", output_fd=result_folder)
    fr.append(df_powerQ_refined["fr"].values[-1])
    qc.append(df_powerQ_refined["Qc_dia_corr_fixed"].values[-1])



In [ ]:
## After assignment each cavity, get foward analysis
import pandas as pd
assignment = pd.read_json(r"C:\Users\ASUS\Documents\python training\thinfat\result\assignment.json")
plot_multiRes_powerQ_free( output_fd, assignment, output_fd)
plot_multiRes_powerQ_refined( output_fd, assignment, output_fd, ((1e-2,1e6),(1e5,1e7)) )
# ((3e-1,3e5),(1e5,5e6))
pd.DataFrame({"fr":np.array(fr),"Qc":np.array(qc)}).to_csv( f"{output_fd}/Qc.csv" )